In [2]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [102]:
# notes harmony simpultan bass

def arpeggiator(c, es,rhythm, duration_divider):
    chords = []
    for i, seq in enumerate(c):
                
        nota = seq
        nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
        chords.append(nota)

    nota = pitch.Pitch(c.pitches[0])
    
    nota.octave = 4
    nota = note.Note(nota.nameWithOctave)
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    #octave adjust
    #print(chords[-1].pitch.midi - chords[-2].pitch.midi, chords[-1])
    if chords[-2].pitch.midi - chords[-3].pitch.midi >= 6:
        chords[-2].octave = chords[-2].octave + 1
        
    if chords[-1].pitch.midi - chords[-2].pitch.midi >= 12:
        chords[-1].octave = chords[-1].octave -1
        #print("yes", chords[-1], chords[-2])
    
    ## last note repeat the second
    nota = pitch.Pitch(c.pitches[2])
    nota.octave = nota.octave
    nota = note.Note(nota.nameWithOctave)
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    return chords

def showHarmonyBass(note_sequence, melody_seq, rhythm, length, time_meter, duration_divider, arp=False,bass=True, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='melody')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='chords')
    p1.timeSignature = meter.TimeSignature(TS)
    p2 = stream.Part(id='bass')
    p2.timeSignature = meter.TimeSignature(TS)
    
    melody_iterator = 0
    for es, sequences in enumerate(note_sequence):
        
        melody = []
        chords = []
        bass = []
        
        
        ### chords
        chord_seq = [sequences[0]]+sequences[1]
        test_chord = chord.Chord(chord_seq)
        c = chord.Chord(test_chord)
        c.root = sequences[0]
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        #c = c.sortDiatonicAscending() # for better voice leadng but not real!
        if es % 2 == 0: # rotating clock counterclock positions
            #c = c.semiClosedPosition(inPlace=False,forceOctave=4)
            pass
        else:
            #c = c.closedPosition(inPlace=False,forceOctave=4)
            pass
        if drop2:
            try:
                #c.pitches[0].octave = c.pitches[0].octave-1 # drop2
                #c.pitches[-2].octave = c.pitches[-2].octave+2 # drop4
                pass
            except:
                pass
        
        if arp:
            chords =  arpeggiator(c, es, rhythm, duration_divider)
            #chords.append(nota)
            
        else:
            chords.append(c)
            
        #MELODY
        # get chord durations and apply on melody (for test purpose)
        '''dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(dur)
        nota = pitch.Pitch(melody_seq[es])
        nota.octave = nota.octave
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur/2
        melody.append(nota)
        #melody 2nd tone
        ###'''
        dur = 0
        for d in chords:
                dur = dur + d.duration.quarterLength
        
        if len(melody_seq) >= len(note_sequence):
            l = round(len(melody_seq) / len(note_sequence))
            #print(l)
            for m in range(l):
                #print(dur, melody_iterator,m, dur/l, l, len(note_sequence), len(melody_seq))
                nota = pitch.Pitch(melody_seq[melody_iterator])
                nota.octave = nota.octave
                nota = note.Note(nota.nameWithOctave)
                nota.duration.quarterLength = dur/l
                melody.append(nota)
                melody_iterator+=1
        
        
        
        #BASS
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(melody_seq, es)
        nota = pitch.Pitch(sequences[0])
        nota.octave = 2
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur
        bass.append(nota)
        
        
        
        p0.append(melody)

        p1.append(chords)
        p2.append(bass)
    
    s.insert(0, p0)
    s.insert(0, p1)
    s.insert(0, p2)
    s = s.transpose(0)
    return s


([3, 1, 2, 2, 1, 3], [3, 1, 2, 1, 1, 1, 1, 2, 1, 3])

In [82]:
# notes harmony 
def showHarmony(note_sequence, time_meter, duration_divider, arp=False, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
   
    p1 = stream.Part(id='part0')
    p1.timeSignature = meter.TimeSignature(TS)
    
    for es, sequences in enumerate(note_sequence):
        chords = []
        
        c = chord.Chord(sequences)
        #c.root = sequences[0]
        #c.octave = 4
        c = c.sortDiatonicAscending()
        c = c.semiClosedPosition(forceOctave=4)
        c.duration.quarterLength = c.duration.quarterLength * duration_divider
        #c.pitches[0].octave = c.pitches[0].octave-2 # drop2
        c.pitches[2].octave = c.pitches[2].octave-1 # drop2
        #c = c.sortDiatonicAscending() # for better voice leadng but not real!
        chords.append(c)
        
        p1.append(chords)
    
   
    s.insert(0, p1)
    s = s.transpose(0)
    return s


In [5]:
# useful functions
# get original combinations by sum
l = _sr_.resultant
ts = 3

def get_original_combinaions_by_sum(l, ts):
    a = []
    for i in range(len(l)):
        for c in itertools.combinations(l, i):
            if sum(c) == ts and list(c) not in a:
                a.append(list(c))
    return a

ps = get_original_combinaions_by_sum(l, ts)
print(ps)
p = list(itertools.permutations(ps))
random.choice(p)
#####
result = sr.resultant
fraction = 2/4
SR.slice_by_fraction(result, fraction)
# random attack combinations!
SR.get_random_attack_combination(_sr_.resultant)
###

[[3], [1, 2], [2, 1], [1, 1, 1]]


[4, 2, 1, 8, 4]

In [105]:
# corpus analyzer plus converter CHORDS
SPG = SP.PitchGroup()

a = corpus.getBachChorales()
ch = a[11]
score = corpus.parse(ch)
excerpt = score.measures(1, 12)

#convert stream to chords and make expansion
original_chords = []
melody = []
chfy = excerpt.chordify()
for c in chfy.flat.getElementsByClass(chord.Chord):
    #print(c.pitches)
    try:
        temp_pitches = []
        original_chords.append(c)
        for p in c.pitches:
            temp_pitches.append(p.name)
            #print(p)

        harm = harmony.chordSymbolFromChord(c)
        sc1 = scale.MajorScale()
        sc1 = sc1.derive(harm)
        scale_names = [str(p.name) for p in sc1.pitches[:-1]]

        #print(sc1, scale_names)
        note_sequence_all_expansions = SPG.translate_notes_to_expansions(temp_pitches, scale_names)
        melody.append(note_sequence_all_expansions[4])
        #print(note_sequence_all_expansions)
    except:
        pass
    

    
#convert exansions to chords
chords = []
for i, tone_group in enumerate(melody):
    c = chord.Chord(tone_group)
    c.duration.quarterLength = original_chords[i].duration.quarterLength
    chords.append(c)

#chfy.show('musicxml')




s = showHarmony(chords, (4,4), 4, drop2=True)


s.show('musicxml')

In [104]:
# corpus analyzer plus converter MELODY
SPG = SP.PitchGroup()

a = corpus.getBachChorales()
ch = a[11]
score = corpus.parse(ch)
excerpt = score.measures(1, 12)

#convert stream to chords and make expansion
original_chords = []
melody = []
scales= []
chfy = excerpt.chordify()
for c in chfy.flat.getElementsByClass(chord.Chord):
    
    try:
        original_chords.append(c)
        melody.append(c.root().name)
        print(c.root().name)
        harm = harmony.chordSymbolFromChord(c)
        sc1 = scale.MajorScale()
        sc1 = sc1.derive(harm)
        print(sc1)
        scale_names = [str(p.name) for p in sc1.pitches[:-1]]
        scales.append(scale_names)
    except:
        pass
        
    
    

    
#convert exansions to chords
chords = []
for i, tone_group in enumerate(melody):
    #c = chord.Chord(tone_group)
    #c.duration.quarterLength = original_chords[i].duration.quarterLength
    #chords.append(c)
    #print(tone_group)
    pass
#chfy.show('musicxml')


bass = melody
print(bass)
scales = scales
seq_rhythm = [1]


voice_amount = 4 # for now because of complicated fake arpeggiator + melody stuff!
theme_expansion_amount = 5
scale_expansion_amount = 1 # 1st order expansion for harmonizing is according to schillinger default (in thirds)

whole_seq = SP.get_whole_sequence_bass(bass, scales, voice_amount, theme_expansion_amount, scale_expansion_amount)
merged_seq = list(itertools.chain(*whole_seq)) # convert to [note1[harmony1], ... noteN[harmonyN]]

## Simple fake MELODY
melody = []
for i, m in enumerate(merged_seq):
    melody.append(random.choice([m[1][0],m[1][1]]))
    if i == len(merged_seq)-1:
        melody[-1] = m[0]

#### TO Notes ->

s = showHarmonyBass(merged_seq, melody, seq_rhythm, len(merged_seq)*2, (3,4), 2, arp=True, bass=False, drop2=True)

s.show('musicxml')




#s = showHarmony(chords, (4,4), 4, drop2=True)


#s.show('musicxml')

G
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
C
<music21.scale.MajorScale G major>
D
<music21.scale.MajorScale A major>
A
<music21.scale.MajorScale G major>
F#
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale B- major>
D
<music21.scale.MajorScale A major>
G
<music21.scale.MajorScale G major>
E
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
D
<music21.scale.MajorScale A major>
G
<music21.scale.MajorScale G major>
A
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
D
<music21.scale.MajorScale G major>
D
<music21.scale.MajorScale G major>
E
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
G
A
<music21.scale.MajorScale G major>
D
<music21.scale.MajorScale A major>
D
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G major>
G
<music21.scale.MajorScale G majo

In [691]:
# tests
test_scale = ["C4", "D4", "E4", "F4", "G4", "A4", "B4"]
SPG = SP.PitchGroup()
scale_expansion = SPG.expansions(test_scale)
C7 = scale_expansion[0]
C3 = scale_expansion[4]
C5 = scale_expansion[2]
diatonic_cycle = SP.diatonic_cicles_with_coeficients(test_scale,[C7,C5,C3,C7,C5,C3],[3,1,3,1,3,1])
#print(diatonic_cycle, len(diatonic_cycle))
